In [1]:
import pandas as pd

file_path2='/Users/gimhagyeong/test/B0005_with_lowess_features.csv'

df=pd.read_csv(file_path2)

In [2]:
df.columns

Index(['Voltage_measured', 'Current_measured', 'Temperature_measured',
       'Current_load', 'Voltage_load', 'cycle_idx', 'Voltage_measured_smooth',
       'Voltage_measured_residual', 'Voltage_measured_trend',
       'Current_measured_smooth', 'Current_measured_residual',
       'Current_measured_trend', 'Temperature_measured_smooth',
       'Temperature_measured_residual', 'Temperature_measured_trend',
       'Current_load_smooth', 'Current_load_residual', 'Current_load_trend',
       'Voltage_load_smooth', 'Voltage_load_residual', 'Voltage_load_trend'],
      dtype='object')

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
import plotly.graph_objects as go


# 1. Train/Validation/Test 분할 (6:2:2)

# 고유 cycle 개수
total_cycles = df['cycle_idx'].nunique()

# 6:2:2 split 기준 cycle 개수
train_cycles = int(total_cycles * 0.6)
val_cycles = int(total_cycles * 0.8)  # train + val

# 정렬된 cycle 리스트
cycle_list = sorted(df['cycle_idx'].unique())

# 기준 cycle 번호
train_threshold_cycle = cycle_list[train_cycles - 1]
val_threshold_cycle = cycle_list[val_cycles - 1]

print(f"총 Cycle 수: {total_cycles}")
print(f"Train Cycles: 0 ~ {train_threshold_cycle} ({train_cycles}개)")
print(f"Validation Cycles: {train_threshold_cycle+1} ~ {val_threshold_cycle} ({val_cycles - train_cycles}개)")
print(f"Test Cycles: {val_threshold_cycle+1} ~ {cycle_list[-1]} ({total_cycles - val_cycles}개)")

# 1. Train / Validation / Test split
train_df = df[df['cycle_idx'] <= train_threshold_cycle].copy()
val_df = df[(df['cycle_idx'] > train_threshold_cycle) & (df['cycle_idx'] <= val_threshold_cycle)].copy()
test_df = df[df['cycle_idx'] > val_threshold_cycle].copy()

print(f"\nTrain size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")


# 2. Feature 준비

features1 = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load', 'Voltage_measured_smooth',
    'Voltage_measured_residual', 'Voltage_measured_trend',
    'Current_measured_smooth', 'Current_measured_residual',
    'Current_measured_trend', 'Temperature_measured_smooth',
    'Temperature_measured_residual', 'Temperature_measured_trend',
    'Current_load_smooth', 'Current_load_residual', 'Current_load_trend',
    'Voltage_load_smooth', 'Voltage_load_residual', 'Voltage_load_trend'
]

X_train = train_df[features1]
X_val = val_df[features1]
X_test = test_df[features1]


# 3. Scaler: Train만으로 학습

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\n Scaler 학습 완료 (Train 기준)")


# 4. LOF: Train만으로 학습

lof = LocalOutlierFactor(
    n_neighbors=30,
    contamination='auto',
    novelty=True
)

lof.fit(X_train_scaled)

print("LOF 모델 학습 완료 (Train 기준)")


# 5. 점수 계산 (Train/Validation/Test)

train_scores = -lof.negative_outlier_factor_
val_scores = -lof.score_samples(X_val_scaled)
test_scores = -lof.score_samples(X_test_scaled)

print(f"\n LOF 점수 범위:")
print(f"   Train: {train_scores.min():.4f} ~ {train_scores.max():.4f}")
print(f"   Validation: {val_scores.min():.4f} ~ {val_scores.max():.4f}")
print(f"   Test: {test_scores.min():.4f} ~ {test_scores.max():.4f}")


# 6. Validation으로 Quantile 기반 Threshold 찾기

# 여러 quantile 시도
quantiles = [0.99, 0.995, 0.999]

print("\n" + "="*70)
print(" Quantile 기반 Threshold 비교 (Validation 기준)")
print("="*70)

quantile_results = []

for q in quantiles:
    # Threshold 계산
    thr = np.quantile(val_scores, q)
    
    # 각 데이터셋의 이상치 개수
    val_anom = (val_scores >= thr).sum()
    test_anom = (test_scores >= thr).sum()
    
    quantile_results.append({
        'quantile': q,
        'threshold': thr,
        'val_count': val_anom,
        'val_pct': val_anom / len(val_df) * 100,
        'test_count': test_anom,
        'test_pct': test_anom / len(test_df) * 100
    })
    
    print(f"\nQuantile {q:.3f} (상위 {(1-q)*100:.1f}%):")
    print(f"   Threshold: {thr:.4f}")
    print(f"   Validation 이상치개수/비율: {val_anom}개 ({val_anom/len(val_df)*100:.2f}%)")
    print(f"   Test 이상치개수/비율: {test_anom}개 ({test_anom/len(test_df)*100:.2f}%)")





총 Cycle 수: 168
Train Cycles: 0 ~ 352 (100개)
Validation Cycles: 353 ~ 483 (34개)
Test Cycles: 484 ~ 614 (34개)

Train size: 29361
Validation size: 10664
Test size: 10260

 Scaler 학습 완료 (Train 기준)
LOF 모델 학습 완료 (Train 기준)

 LOF 점수 범위:
   Train: 0.9388 ~ 20.5561
   Validation: 0.9384 ~ 4.2310
   Test: 0.9356 ~ 7.6131

 Quantile 기반 Threshold 비교 (Validation 기준)

Quantile 0.990 (상위 1.0%):
   Threshold: 2.3324
   Validation 이상치개수/비율: 107개 (1.00%)
   Test 이상치개수/비율: 2771개 (27.01%)

Quantile 0.995 (상위 0.5%):
   Threshold: 2.5594
   Validation 이상치개수/비율: 54개 (0.51%)
   Test 이상치개수/비율: 1944개 (18.95%)

Quantile 0.999 (상위 0.1%):
   Threshold: 3.5380
   Validation 이상치개수/비율: 11개 (0.10%)
   Test 이상치개수/비율: 54개 (0.53%)


In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

selected_quantile = 0.999
threshold = np.quantile(val_scores, selected_quantile)


# 1. 점수 & split 정보 붙이기
train_plot = train_df.copy()
val_plot   = val_df.copy()
test_plot  = test_df.copy()

train_plot['score'] = train_scores
val_plot['score']   = val_scores
test_plot['score']  = test_scores

train_plot['split'] = 'train'
val_plot['split']   = 'val'
test_plot['split']  = 'test'

full = pd.concat([train_plot, val_plot, test_plot], ignore_index=True)

# 이상치 여부
full['is_anom'] = full['score'] >= threshold

# 2. cycle 단위 요약 (예: max score, 이상치 비율)
cycle_summary = (
    full.groupby(['split', 'cycle_idx'])
        .agg(
            max_score=('score', 'max'),
            mean_score=('score', 'mean'),
            frac_anom=('is_anom', 'mean')   # 해당 cycle 내 이상치 비율
        )
        .reset_index()
)

# cycle에 하나라도 이상치 있으면 True
cycle_summary['has_anom'] = cycle_summary['frac_anom'] > 0
cycle_summary['has_anom'] = cycle_summary['has_anom'].map({True: 'anomaly cycle', False: 'normal cycle'})



In [7]:
cycle_anomaly = (
    full.groupby('cycle_idx')['score']
        .agg(mean_score='mean', 
             median_score='median', 
             max_score='max')
        .reset_index()
)

In [8]:
raw_discharge_df=pd.read_csv('/Users/gimhagyeong/test/B0005_discharge.csv')

In [ ]:
capacity_cycle = (
    raw_discharge_df
      .groupby('cycle_idx')['Capacity']
      .agg('max')               # 또는 'first'
      .reset_index()
)

In [9]:
import numpy as np
import pandas as pd

df = cycle_anomaly.copy()

# 0) split 컬럼이 없으면 cycle_idx로 만들어주기
# (이미 split이 있으면 이 블록은 건너뛰어도 됨)
if 'split' not in df.columns:
    df['split'] = np.select(
        [
            (df['cycle_idx'] >= 0) & (df['cycle_idx'] <= 352),
            (df['cycle_idx'] >= 353) & (df['cycle_idx'] <= 483),
            (df['cycle_idx'] >= 484) & (df['cycle_idx'] <= 614),
        ],
        ['train', 'val', 'test'],
        default='other'
    )

# 1) 어떤 score로 HI 만들지 선택 (b0005면 보통 mean_score 추천)
score_col = 'mean_score'   # 'median_score'로 바꿔도 됨

train_mask = df['split'].eq('train')
val_mask   = df['split'].eq('val')
test_mask  = df['split'].eq('test')

# 2) train 기준 robust 정규화 파라미터
train_scores = df.loc[train_mask, score_col].astype(float)

med = train_scores.median()
q1  = train_scores.quantile(0.25)
q3  = train_scores.quantile(0.75)
iqr = (q3 - q1) if (q3 - q1) != 0 else 1e-8

# 3) 정상 대비 이탈(robust z) -> 음수 제거 -> log 압축
df['z_t'] = (df[score_col].astype(float) - med) / iqr
df['s_t'] = df['z_t'].clip(lower=0)
df['s_t_log'] = np.log1p(df['s_t'])

# 4) HI(EMA): val, test 각각 0에서 시작
alpha = 0.2
df['HI_ema'] = np.nan

def ema_with_init(arr, alpha, init=0.0):
    hi = np.zeros_like(arr, dtype=float)
    hi[0] = init
    for i in range(1, len(arr)):
        hi[i] = alpha * arr[i] + (1 - alpha) * hi[i-1]
    return hi

# val HI (0에서 시작)
val_arr = df.loc[val_mask, 's_t_log'].to_numpy(dtype=float)
val_hi = ema_with_init(val_arr, alpha, init=0.0)
df.loc[val_mask, 'HI_ema'] = val_hi

# test HI (val 마지막 값에서 이어받기)
test_arr = df.loc[test_mask, 's_t_log'].to_numpy(dtype=float)
init_test = val_hi[-1]  # ⭐ 핵심
test_hi = ema_with_init(test_arr, alpha, init=init_test)
df.loc[test_mask, 'HI_ema'] = test_hi

df


,cycle_idx,mean_score,median_score,max_score,split,z_t,s_t,s_t_log,HI_ema
0,2,1.131456,1.096262,1.654330,train,0.666271,0.666271,0.510588,NaN
1,4,1.117758,1.091529,1.941832,train,0.498523,0.498523,0.404480,NaN
2,6,1.130816,1.055099,5.891674,train,0.658442,0.658442,0.505879,NaN
3,8,1.092035,1.016034,6.126168,train,0.183500,0.183500,0.168476,NaN
4,10,1.109424,1.001969,12.788607,train,0.396465,0.396465,0.333944,NaN
...,...,...,...,...,...,...,...,...,...
163,600,2.167378,2.082073,3.923075,test,13.352795,13.352795,2.663945,2.583054
164,604,2.194079,2.040987,4.048687,test,13.679790,13.679790,2.686472,2.603738
165,608,2.198891,2.069808,3.679316,test,13.738720,13.738720,2.690478,2.621086
166,612,1.773032,1.628673,3.411143,test,8.523392,8.523392,2.253751,2.547619


In [11]:
val_test_df = df[df['cycle_idx'] >= 353].copy().reset_index(drop=True)

In [12]:
val_test_df

,cycle_idx,mean_score,median_score,max_score,split,z_t,s_t,s_t_log,HI_ema
0,356,1.041919,1.024173,1.403090,val,-0.430253,0.000000,0.000000,0.000000
1,360,1.066462,1.041889,1.522469,val,-0.129681,0.000000,0.000000,0.000000
2,364,1.094211,1.058839,3.310529,val,0.210157,0.210157,0.190751,0.038150
3,367,1.301780,1.250749,3.192214,val,2.752161,2.752161,1.322332,0.294986
4,371,1.259828,1.246566,3.650730,val,2.238400,2.238400,1.175079,0.471005
...,...,...,...,...,...,...,...,...,...
63,600,2.167378,2.082073,3.923075,test,13.352795,13.352795,2.663945,2.583054
64,604,2.194079,2.040987,4.048687,test,13.679790,13.679790,2.686472,2.603738
65,608,2.198891,2.069808,3.679316,test,13.738720,13.738720,2.690478,2.621086
66,612,1.773032,1.628673,3.411143,test,8.523392,8.523392,2.253751,2.547619


In [ ]:
val_test_df['HI_slope_ML'] =val_test_df['HI_ema'].diff()
val_test_df['abs_HI_slope_ML'] =val_test_df['HI_slope_ML'].abs()

df['HI_slope_DL'] = df['HI_DL'].diff()
df['abs_HI_slope_DL'] = df['HI_slope_DL'].abs()

df_clean = df.dropna(subset=['abs_HI_slope_ML', 'abs_HI_slope_DL'])


In [ ]:
from scipy.stats import wilcoxon

x = df_clean['abs_HI_slope_ML']
y = df_clean['abs_HI_slope_DL']

stat, p = wilcoxon(x, y)
print(p)


In [ ]:
df['abs_dcap'] = df['capacity'].diff().abs()
df['abs_dcap'].describe()
early = df.query('370<=cycle_idx<=420')['abs_dcap'].dropna()
late  = df.query('420<cycle_idx<=560')['abs_dcap'].dropna()

early = df.query('370<=cycle_idx<=420')['abs_dcap'].dropna()
late  = df.query('420<cycle_idx<=560')['abs_dcap'].dropna()

In [ ]:
merged = cycle_anomaly.merge(capacity_cycle[['cycle_idx','Capacity']], on='cycle_idx')

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ========================================
# Cycle 단위 집계 3가지 방법
# ========================================

print("="*70)
print("🔬 Cycle 단위 집계 전략 비교")
print("="*70)

# 1) Mean
cycle_agg_mean = df.groupby('cycle_idx').agg({
    'Capacity': 'mean',
    'Voltage_load_trend': 'mean',
    'Voltage_load_residual': 'mean',
    'Current_load_trend': 'mean',
    'Current_load_residual': 'mean',
    'mean_score': 'mean'
}).reset_index()

cycle_agg_mean['abs_dcap'] = cycle_agg_mean['Capacity'].diff().abs()
cycle_agg_mean['abs_V_trend'] = cycle_agg_mean['Voltage_load_trend'].diff().abs()
cycle_agg_mean['abs_V_resid'] = cycle_agg_mean['Voltage_load_residual'].diff().abs()
cycle_agg_mean['abs_I_trend'] = cycle_agg_mean['Current_load_trend'].diff().abs()
cycle_agg_mean['abs_I_resid'] = cycle_agg_mean['Current_load_residual'].diff().abs()

# 2) Max
cycle_agg_max = df.groupby('cycle_idx').agg({
    'Capacity': 'mean',  # Capacity는 평균
    'Voltage_load_trend': 'max',
    'Voltage_load_residual': 'max',
    'Current_load_trend': 'max',
    'Current_load_residual': 'max',
    'mean_score': 'mean'
}).reset_index()

cycle_agg_max['abs_dcap'] = cycle_agg_max['Capacity'].diff().abs()
cycle_agg_max['abs_V_trend'] = cycle_agg_max['Voltage_load_trend'].diff().abs()
cycle_agg_max['abs_V_resid'] = cycle_agg_max['Voltage_load_residual'].diff().abs()
cycle_agg_max['abs_I_trend'] = cycle_agg_max['Current_load_trend'].diff().abs()
cycle_agg_max['abs_I_resid'] = cycle_agg_max['Current_load_residual'].diff().abs()

# 3) Median
cycle_agg_median = df.groupby('cycle_idx').agg({
    'Capacity': 'mean',  # Capacity는 평균
    'Voltage_load_trend': 'median',
    'Voltage_load_residual': 'median',
    'Current_load_trend': 'median',
    'Current_load_residual': 'median',
    'mean_score': 'mean'
}).reset_index()

cycle_agg_median['abs_dcap'] = cycle_agg_median['Capacity'].diff().abs()
cycle_agg_median['abs_V_trend'] = cycle_agg_median['Voltage_load_trend'].diff().abs()
cycle_agg_median['abs_V_resid'] = cycle_agg_median['Voltage_load_residual'].diff().abs()
cycle_agg_median['abs_I_trend'] = cycle_agg_median['Current_load_trend'].diff().abs()
cycle_agg_median['abs_I_resid'] = cycle_agg_median['Current_load_residual'].diff().abs()

# ========================================
# Test 구간 비교
# ========================================
test_mean = cycle_agg_mean.query('484 <= cycle_idx <= 614')
test_max = cycle_agg_max.query('484 <= cycle_idx <= 614')
test_median = cycle_agg_median.query('484 <= cycle_idx <= 614')

print("\n[Voltage_load_trend 비교 (Test 구간)]")
print(f"\nMean 방식:")
print(f"  평균: {test_mean['abs_V_trend'].mean():.6f}")
print(f"  분산: {test_mean['abs_V_trend'].var():.8f}")
print(f"  99%:  {test_mean['abs_V_trend'].quantile(0.99):.6f}")

print(f"\nMax 방식:")
print(f"  평균: {test_max['abs_V_trend'].mean():.6f}")
print(f"  분산: {test_max['abs_V_trend'].var():.8f}")
print(f"  99%:  {test_max['abs_V_trend'].quantile(0.99):.6f}")

print(f"\nMedian 방식:")
print(f"  평균: {test_median['abs_V_trend'].mean():.6f}")
print(f"  분산: {test_median['abs_V_trend'].var():.8f}")
print(f"  99%:  {test_median['abs_V_trend'].quantile(0.99):.6f}")

# ========================================
# 시각화 비교
# ========================================
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=test_mean['cycle_idx'],
    y=test_mean['abs_V_trend'],
    mode='lines',
    name='Mean',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=test_max['cycle_idx'],
    y=test_max['abs_V_trend'],
    mode='lines',
    name='Max',
    line=dict(color='red', width=2)
))

fig.add_trace(go.Scatter(
    x=test_median['cycle_idx'],
    y=test_median['abs_V_trend'],
    mode='lines',
    name='Median',
    line=dict(color='green', width=2)
))

fig.update_layout(
    title='Cycle 집계 방법 비교: Voltage_load_trend 변화량',
    xaxis_title='Cycle',
    yaxis_title='|ΔVoltage_load_trend|',
    height=500
)

fig.show()

# ========================================
# Capacity 대비 배수 비교
# ========================================
print("\n" + "="*70)
print("📊 Capacity 대비 배수 (99% 분위수)")
print("="*70)

test_cap = test_mean['abs_dcap']  # Capacity는 동일

methods = {
    'Mean': test_mean,
    'Max': test_max,
    'Median': test_median
}

for method_name, test_data in methods.items():
    q99_cap = test_cap.quantile(0.99)
    q99_volt = test_data['abs_V_trend'].quantile(0.99)
    q99_curr = test_data['abs_I_resid'].quantile(0.99)
    
    print(f"\n[{method_name} 방식]")
    print(f"  Voltage_load_trend: {q99_volt/q99_cap:.1f}배")
    print(f"  Current_load_resid: {q99_curr/q99_cap:.1f}배")

# ========================================
# 노이즈 분석
# ========================================
print("\n" + "="*70)
print("🔍 노이즈 정도 비교")
print("="*70)

# 연속된 cycle 간 변동성 (급변 정도)
for method_name, test_data in methods.items():
    # 변화량의 변화량 (2차 미분 개념)
    volatility = test_data['abs_V_trend'].diff().abs().mean()
    print(f"{method_name:8s} 변동성: {volatility:.6f}")

print("\n→ 변동성 낮을수록 안정적 (Mean < Median < Max)")

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ========================================
# 데이터 준비
# ========================================
df = hi_merged.copy()

# 변화량 계산
df['abs_dcap'] = df['Capacity'].diff().abs()
df['abs_V_trend'] = df['Voltage_load_trend'].diff().abs()
df['abs_V_resid'] = df['Voltage_load_residual'].diff().abs()
df['abs_I_trend'] = df['Current_load_trend'].diff().abs()
df['abs_I_resid'] = df['Current_load_residual'].diff().abs()

# Anomaly 정의
if 'is_anom' not in df.columns:
    threshold = df['mean_score'].quantile(0.90)
    df['is_anom'] = df['mean_score'] > threshold

print("="*80)
print("📊 배터리 열화 패턴 분석: Capacity는 느리고, 전압/전류는 튄다")
print("="*80)

# ========================================
# Test 구간
# ========================================
test_df = df.query('484 <= cycle_idx <= 614').copy()
print(f"\nTest 구간: Cycle 484~614 ({len(test_df)} cycles)")
print("→ 후반 열화 구간 집중 분석")

# ========================================
# 🎯 핵심 1: "Capacity는 느리다" (완만한 변화)
# ========================================
print("\n" + "="*80)
print("📉 증명 1: Capacity는 완만하게 변한다")
print("="*80)

test_cap = test_df['abs_dcap'].dropna()

print("\n[Capacity 변화량 통계]")
print(f"  평균:       {test_cap.mean():.6f}")
print(f"  중앙값:     {test_cap.median():.6f}")
print(f"  표준편차:   {test_cap.std():.6f}")
print(f"  분산:       {test_cap.var():.8f}")

print("\n[Capacity 분위수]")
for q in [0.5, 0.75, 0.90, 0.95, 0.99]:
    print(f"  {int(q*100):2d}%: {test_cap.quantile(q):.6f}")

# Spike Rate (95% 기준)
cap_thr_95 = test_cap.quantile(0.95)
cap_spike_rate = (test_cap > cap_thr_95).mean()
print(f"\n[Capacity Spike Rate]")
print(f"  95% threshold: {cap_thr_95:.6f}")
print(f"  초과 비율:     {cap_spike_rate:.1%}")
print(f"  → 큰 변화가 드물다 (완만)")

# ========================================
# 🎯 핵심 2: "전압/전류는 튄다" (급변)
# ========================================
print("\n" + "="*80)
print("⚡ 증명 2: 전압/전류는 후반부에서 순간적으로 튄다")
print("="*80)

# 4가지 후보
candidates = {
    'Voltage_load_trend': {
        'data': test_df['abs_V_trend'].dropna(),
        'label': 'SHAP 1위',
        'col': 'Voltage_load_trend'
    },
    'Voltage_load_residual': {
        'data': test_df['abs_V_resid'].dropna(),
        'label': 'Voltage Residual',
        'col': 'Voltage_load_residual'
    },
    'Current_load_trend': {
        'data': test_df['abs_I_trend'].dropna(),
        'label': 'JSD 2위',
        'col': 'Current_load_trend'
    },
    'Current_load_residual': {
        'data': test_df['abs_I_resid'].dropna(),
        'label': 'JSD 1위',
        'col': 'Current_load_residual'
    }
}

comparison_data = []

for name, info in candidates.items():
    data = info['data']
    
    print(f"\n[{name}]")
    print(f"   → {info['label']}")
    
    mean_val = data.mean()
    var_val = data.var()
    q99_val = data.quantile(0.99)
    
    print(f"\n  평균:     {mean_val:.6f} (Cap의 {mean_val/test_cap.mean():5.1f}배)")
    print(f"  분산:     {var_val:.8f} (Cap의 {var_val/test_cap.var():5.1f}배)")
    print(f"  99%:      {q99_val:.6f} (Cap의 {q99_val/test_cap.quantile(0.99):5.1f}배)")
    
    # Spike Rate
    thr_95 = data.quantile(0.95)
    spike_rate = (data > thr_95).mean()
    print(f"\n  95% threshold: {thr_95:.6f}")
    print(f"  Spike Rate:    {spike_rate:.1%} (Cap의 {spike_rate/cap_spike_rate:.1f}배)")
    
    comparison_data.append({
        'Variable': name,
        'Label': info['label'],
        'Mean': mean_val,
        'Mean_Ratio': mean_val/test_cap.mean(),
        'Var': var_val,
        'Var_Ratio': var_val/test_cap.var(),
        'Q99': q99_val,
        'Q99_Ratio': q99_val/test_cap.quantile(0.99),
        'Spike_Rate': spike_rate,
        'Spike_Ratio': spike_rate/cap_spike_rate
    })

# ========================================
# 🎯 핵심 비교표: Capacity vs 전압/전류
# ========================================
print("\n" + "="*80)
print("🎯 핵심 비교: Capacity는 느리고, 전압/전류는 튄다")
print("="*80)

comparison_df = pd.DataFrame(comparison_data)

print("\n[Capacity 대비 배수 비교]")
print(f"\n{'변수':<25} {'평균배수':>8} {'분산배수':>8} {'99%배수':>8} {'Spike배수':>8}")
print("-" * 80)
print(f"{'Capacity (기준)':<25} {'1.0x':>8} {'1.0x':>8} {'1.0x':>8} {'1.0x':>8}")
for _, row in comparison_df.iterrows():
    print(f"{row['Variable']:<25} "
          f"{row['Mean_Ratio']:>7.1f}x "
          f"{row['Var_Ratio']:>7.1f}x "
          f"{row['Q99_Ratio']:>7.1f}x "
          f"{row['Spike_Ratio']:>7.1f}x")

# 최대값 강조
max_var = comparison_df.loc[comparison_df['Var_Ratio'].idxmax()]
max_q99 = comparison_df.loc[comparison_df['Q99_Ratio'].idxmax()]
max_spike = comparison_df.loc[comparison_df['Spike_Ratio'].idxmax()]

print(f"\n[핵심 발견]")
print(f"  분산 최대:      {max_var['Variable']} ({max_var['Var_Ratio']:.1f}배)")
print(f"  99% 최대:       {max_q99['Variable']} ({max_q99['Q99_Ratio']:.1f}배)")
print(f"  Spike Rate 최대: {max_spike['Variable']} ({max_spike['Spike_Ratio']:.1f}배)")

# ========================================
# Mann-Whitney U Test
# ========================================
print("\n" + "="*80)
print("📈 통계 검정: Anomaly vs Normal")
print("="*80)

mw_results = []

for name, info in candidates.items():
    col = info['col']
    
    test_clean = test_df.dropna(subset=[col, 'is_anom'])
    
    if len(test_clean) > 0:
        normal = test_clean[~test_clean['is_anom']][col]
        anomaly = test_clean[test_clean['is_anom']][col]
        
        if len(normal) > 0 and len(anomaly) > 0:
            u_stat, p_value = mannwhitneyu(normal, anomaly, alternative='less')
            
            print(f"\n[{name}]")
            print(f"  Normal 평균:  {normal.mean():.6f}")
            print(f"  Anomaly 평균: {anomaly.mean():.6f}")
            print(f"  차이:         {anomaly.mean() - normal.mean():.6f}")
            print(f"  P-value:      {p_value:.6f}", end="")
            
            if p_value < 0.05:
                print(" ✅ 유의함")
                sig = "유의"
            else:
                print(" ❌ 비유의")
                sig = "비유의"
            
            mw_results.append({
                'Variable': name,
                'P_value': p_value,
                'Significance': sig
            })

# ========================================
# 최종 증명 문장
# ========================================
print("\n" + "="*80)
print("📝 최종 증명 문장")
print("="*80)

# 최고 변수 선택
top_var = comparison_df.loc[comparison_df['Q99_Ratio'].idxmax()]

summary = f"""
Test 구간(후반 열화 구간, Cycle 484~614)에서:

1️⃣ Capacity는 완만하게 변한다
   - 평균 변화량: {test_cap.mean():.6f}
   - 분산: {test_cap.var():.8f}
   - 99% 분위수: {test_cap.quantile(0.99):.6f}
   - Spike Rate: {cap_spike_rate:.1%}
   → 느리고 안정적인 감소 패턴

2️⃣ 전압/전류는 후반부에서 순간적으로 튄다
   - 최고: {top_var['Variable']} ({top_var['Label']})
   - 평균 {top_var['Mean_Ratio']:.1f}배, 분산 {top_var['Var_Ratio']:.1f}배, 
     99% 분위수 {top_var['Q99_Ratio']:.1f}배, Spike Rate {top_var['Spike_Ratio']:.1f}배
   → Capacity 대비 압도적으로 큰 변화

3️⃣ 통계 검정 결과
   - {len([r for r in mw_results if r['Significance']=='유의'])}개 변수 모두 
     Anomaly 그룹에서 유의하게 높음 (Mann-Whitney U-test, p<0.05)

결론:
후반 열화 구간에서 Capacity는 완만히 감소하는 반면,
전압과 전류는 국소적으로 급변하며 이상 탐지의 핵심 신호가 된다.
"""

print(summary)

# ========================================
# 시각화
# ========================================
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Capacity 변화량 분포', 
        'Capacity vs 전압/전류 Box Plot',
        'Voltage_load_trend', 
        'Voltage_load_residual',
        'Current_load_trend', 
        'Current_load_residual'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.12,
    specs=[[{"type": "histogram"}, {"type": "box"}],
           [{"type": "histogram"}, {"type": "histogram"}],
           [{"type": "histogram"}, {"type": "histogram"}]]
)

# Row 1, Col 1: Capacity 히스토그램
fig.add_trace(
    go.Histogram(x=test_cap, name='Capacity', nbinsx=50, 
                 marker_color='green', showlegend=False),
    row=1, col=1
)

# Row 1, Col 2: Box Plot 비교
fig.add_trace(
    go.Box(y=test_cap, name='Capacity', marker_color='green'),
    row=1, col=2
)
for name, info in candidates.items():
    fig.add_trace(
        go.Box(y=info['data'], name=name.replace('_', ' '), 
               marker_color='red' if 'Voltage' in name else 'blue'),
        row=1, col=2
    )

# Row 2-3: 4개 변수 히스토그램
positions = [(2,1), (2,2), (3,1), (3,2)]
colors = ['blue', 'lightblue', 'red', 'lightcoral']

for (name, info), (row, col), color in zip(candidates.items(), positions, colors):
    fig.add_trace(
        go.Histogram(x=info['data'], name=name, nbinsx=50,
                     marker_color=color, showlegend=False),
        row=row, col=col
    )

fig.update_layout(
    height=1200,
    title_text="Capacity는 느리고, 전압/전류는 튄다 (Test 구간)",
    showlegend=True
)

fig.show()

# ========================================
# 추가 시각화: 배수 비교 바 차트
# ========================================
fig2 = go.Figure()

metrics = ['Mean_Ratio', 'Var_Ratio', 'Q99_Ratio', 'Spike_Ratio']
metric_names = ['평균 배수', '분산 배수', '99% 배수', 'Spike Rate 배수']

for metric, metric_name in zip(metrics, metric_names):
    fig2.add_trace(go.Bar(
        name=metric_name,
        x=comparison_df['Variable'],
        y=comparison_df[metric],
        text=comparison_df[metric].apply(lambda x: f'{x:.1f}x'),
        textposition='outside'
    ))

fig2.update_layout(
    title='Capacity 대비 전압/전류 변화 배수 비교',
    xaxis_title='변수',
    yaxis_title='배수 (Capacity = 1.0)',
    barmode='group',
    height=600
)

fig2.show()

# ========================================
# 저장
# ========================================
# 비교표
comparison_summary = pd.DataFrame({
    'Variable': ['Capacity'] + list(comparison_df['Variable']),
    'Mean': [test_cap.mean()] + list(comparison_df['Mean']),
    'Var': [test_cap.var()] + list(comparison_df['Var']),
    'Q99': [test_cap.quantile(0.99)] + list(comparison_df['Q99']),
    'Spike_Rate': [cap_spike_rate] + list(comparison_df['Spike_Rate'])
})

comparison_summary.to_csv('capacity_vs_voltage_current_comparison.csv', index=False)

# 배수 비교
comparison_df.to_csv('capacity_ratio_comparison.csv', index=False)

# Mann-Whitney 결과
if mw_results:
    pd.DataFrame(mw_results).to_csv('mannwhitney_test_results.csv', index=False)

print("\n✅ 저장 완료:")
print("   - capacity_vs_voltage_current_comparison.csv")
print("   - capacity_ratio_comparison.csv")
print("   - mannwhitney_test_results.csv")

print("\n" + "="*80)
print("🎯 분석 완료!")
print("="*80)